In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, mean_squared_error
from xgboost import XGBClassifier, XGBRegressor
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import LabelEncoder

Load the Dataset

In [3]:
data = pd.read_csv('2021_Green_Taxi_Trip_Data.csv')

C:\Users\Huawei\AppData\Local\Temp\ipykernel_1284\1318034804.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('2021_Green_Taxi_Trip_Data.csv')


Remove Unnecessary Columns

In [4]:
data = data.drop(columns=['ehail_fee', 'improvement_surcharge', 'store_and_fwd_flag', 'RatecodeID', 'congestion_surcharge', 'fare_amount', 'extra', 'mta_tax'])

Check number of nulls

In [5]:
data.isnull().sum()

VendorID                 249115
lpep_pickup_datetime          0
lpep_dropoff_datetime         0
PULocationID                  0
DOLocationID                  0
passenger_count          412434
trip_distance                 0
tip_amount                    0
tolls_amount                  0
total_amount                  0
payment_type             412434
trip_type                412434
dtype: int64

Check payment types when tip amount > 0

In [6]:
filtered_data = data[data['tip_amount'] > 0]
filtered_data.groupby('payment_type').size()

payment_type
1.0    298760
3.0         1
dtype: int64

So we consider payment type 1.0 to be a credit card. If a tip was given, assume payment was made by credit card(1.0). (We have just one row with different payment type (3.0), so we change all of the rows with the tip into payment type = 1.0)

In [7]:
data.loc[(data['tip_amount'] > 0), 'payment_type'] = 1.0

Check trip types when passenger count > 4

In [10]:
filtered_data = data[data['passenger_count'] > 4]
filtered_data.groupby('trip_type').size()

trip_type
1.0    22695
2.0      382
dtype: int64

So we consider trip type 1.0 to be dispatch. (We have just 1.66% of rows with different trip type (2.0), so we change all of the rows with the tip into trip type = 1.0)

Check Payment Types

In [13]:
data.groupby('payment_type').size()

payment_type
1.0    461268
2.0    259274
3.0      3891
4.0      1091
5.0         7
dtype: int64

number of payment type = 5.0 is 7, which is so little, we could consider payment type = 5.0 as unknown.

Classify Vans and Set Trip Type

In [14]:
data.loc[data['passenger_count'] > 4, 'trip_type'] = 1.0
data.loc[data['passenger_count'] > 4, 'payment_type'] = 5.0

Remove Rows with Missing VnedorID

In [15]:
data = data.dropna(subset=['VendorID'])

Filter by Year (2021 Only)

In [16]:
data['pickup_datetime'] = pd.to_datetime(data['lpep_pickup_datetime'], format='%m/%d/%Y %I:%M:%S %p')
data = data[data['pickup_datetime'].dt.year == 2021]

In [17]:
data.isnull().mean()


VendorID                 0.000000
lpep_pickup_datetime     0.000000
lpep_dropoff_datetime    0.000000
PULocationID             0.000000
DOLocationID             0.000000
passenger_count          0.199264
trip_distance            0.000000
tip_amount               0.000000
tolls_amount             0.000000
total_amount             0.000000
payment_type             0.163446
trip_type                0.199264
pickup_datetime          0.000000
dtype: float64

In [ ]:
data['lpep_pickup_datetime'] = pd.to_datetime(data['lpep_pickup_datetime'], errors='coerce')
data['lpep_dropoff_datetime'] = pd.to_datetime(data['lpep_dropoff_datetime'], errors='coerce')
data['trip_duration'] = (data['lpep_dropoff_datetime'] - data['lpep_pickup_datetime']).dt.total_seconds() / 60

C:\Users\Huawei\AppData\Local\Temp\ipykernel_1284\3327706167.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['lpep_pickup_datetime'] = pd.to_datetime(data['lpep_pickup_datetime'], errors='coerce')


KeyboardInterrupt: 

In [20]:
from sklearn.ensemble import RandomForestClassifier
def select_features_for_passenger_count(data, target_column='passenger_count', n_features=4):
    x_pass = data.drop(columns=[target_column, 'payment_type', 'trip_type', 'lpep_pickup_datetime', 'lpep_dropoff_datetime'])
    y_pass = data[target_column].dropna()  
    x_pass = x_pass.loc[y_pass.index]  
    
    
    model = RandomForestClassifier(n_estimators=50, random_state=0)
    model.fit(x_pass, y_pass)
    
    
    feature_importances = pd.Series(model.feature_importances_, index=x_pass.columns)
    
    
    selected_features = feature_importances.nlargest(n_features).index.tolist()
    return selected_features

selected_features = select_features_for_passenger_count(data)
print("Selected features for predicting passenger_count:", selected_features)

DTypePromotionError: The DType <class 'numpy.dtypes.DateTime64DType'> could not be promoted by <class 'numpy.dtypes.Float64DType'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Int64DType'>, <class 'numpy.dtypes.Int64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.DateTime64DType'>)